In [ ]:
import os
from google.colab import userdata

# Note: `userdata.get` is a Colab API. If you're not using Colab, set the env
# vars as appropriate for your system.

os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')
os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')

In [ ]:
# Install Keras 3 last. See https://keras.io/getting_started/ for more details.
!pip install -q -U keras-nlp
!pip install -q -U "keras>=3"
!pip install -U keras-hub


In [ ]:
os.environ["KERAS_BACKEND"] = "jax"  # Or "torch" or "tensorflow".
# Avoid memory fragmentation on JAX backend.
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"

In [ ]:
import keras
import keras_nlp

In [ ]:
!wget -O yoto.jsonl https://huggingface.co/datasets/Vineeth11/Equipd/resolve/main/yoto.jsonl

--2025-01-08 08:11:09--  https://huggingface.co/datasets/Vineeth11/Equipd/resolve/main/yoto.jsonl
Resolving huggingface.co (huggingface.co)... 18.164.174.23, 18.164.174.118, 18.164.174.55, ...
Connecting to huggingface.co (huggingface.co)|18.164.174.23|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33317 (33K) [text/plain]
Saving to: ‘yoto.jsonl’

yoto.jsonl          100%[===================>]  32.54K  --.-KB/s    in 0.007s  

2025-01-08 08:11:10 (4.33 MB/s) - ‘yoto.jsonl’ saved [33317/33317]



In [ ]:
import json
data = []
with open("yoto.jsonl") as file:
    for line in file:
        features = json.loads(line)
        # Filter out examples with context, to keep it simple.
        if features["context"]:
            continue
        # Format the entire example as a single string.
import json
data = []
with open("yoto.jsonl") as file:
    for line in file:
        features = json.loads(line)
        # Filter out examples with context, to keep it simple.
        if features["context"]:
            continue
        # Format the entire example as a single string.
        try:
            template = "Instruction:\n{instruction}\n\nResponse:\n{response}"
            data.append(template.format(**features))
        except KeyError:
            # Handle cases where 'response' is missing (e.g., skip or print a warning)
            print(f"Warning: 'response' key missing in line: {line.strip()}")


# Only use 100 training examples, to keep it fast.
data = data[:100]


In [ ]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_2b_en")
gemma_lm.summary()

In [ ]:
# Define the template outside the with block to make it accessible
template = "Instruction:\n{instruction}\n\nResponse:\n{response}"

prompt = template.format(
    instruction="What is Machine Learning?",
    response="",
)
sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt, max_length=256))

In [ ]:
prompt = template.format(
    instruction="What are the types of classification?",
    response="",
)
print(gemma_lm.generate(prompt, max_length=256))

In [ ]:
# Enable LoRA for the model and set the LoRA rank to 4.
gemma_lm.backbone.enable_lora(rank=4)
gemma_lm.summary()

In [ ]:
prompt = template.format(
    instruction="What is Deep Learning?.",
    response="",
)
print(gemma_lm.generate(prompt, max_length=300))

In [ ]:
# Limit the input sequence length to 256 (to control memory usage).
gemma_lm.preprocessor.sequence_length = 256
# Use AdamW (a common optimizer for transformer models).
optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
)
# Exclude layernorm and bias terms from decay.
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)
gemma_lm.fit(data, epochs=1, batch_size=1)

In [ ]:
prompt = template.format(
    instruction="What is Deep Learning?",
    response="",
)
sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt, max_length=256))

In [ ]:
prompt = template.format(
    instruction="What is Deep Learning?",
    response="",
)
print(gemma_lm.generate(prompt, max_length=256))

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

# Example data (replace with your actual metrics)
epochs = np.arange(1, 101)  # Epochs from 1 to 100
train_acc = np.linspace(0.1, 0.9, 100)  # Example training accuracy
val_acc = np.linspace(0.15, 0.85, 100)  # Example validation accuracy

train_loss = np.linspace(3.5, 2.5, 100) + np.random.randn(100) * 0.1  # Example training loss
val_loss = np.linspace(3.6, 2.6, 100) + np.random.randn(100) * 0.1  # Example validation loss

train_grad_norm = np.linspace(9, 4, 100) + np.random.randn(100) * 0.5  # Example gradient norm (train)
val_grad_norm = np.linspace(9, 4.2, 100) + np.random.randn(100) * 0.5  # Example gradient norm (val)

# Plot Accuracy
plt.figure(figsize=(12, 8))

plt.subplot(3, 1, 1)
plt.plot(epochs, train_acc, label="training", color="purple", linewidth=2)
plt.plot(epochs, val_acc, label="validation", color="violet", linewidth=2, alpha=0.7)
plt.title("Gemma 2B Model Training and Validation Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.grid()

# Plot Gradient Norm
plt.subplot(3, 1, 2)
plt.plot(epochs, train_grad_norm, label="training", color="blue", linewidth=2)
plt.plot(epochs, val_grad_norm, label="validation", color="lightblue", linewidth=2, alpha=0.7)
plt.title("Gemma 2B Model Gradient Norm Training and Validation")
plt.xlabel("Epochs")
plt.ylabel("grad_norm")
plt.legend()
plt.grid()

# Plot Loss
plt.subplot(3, 1, 3)
plt.plot(epochs, train_loss, label="training", color="orange", linewidth=2)
plt.plot(epochs, val_loss, label="validation", color="gold", linewidth=2, alpha=0.7)
plt.title("Gemma 2B Model Training and Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.grid()

plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Data for the plot
epochs = np.linspace(0, 100, 100)  # x-axis (0 to 100 epochs)
learning_rate = np.full_like(epochs, 2e-4)  # y-axis (constant learning rate 2e-4)

# Plot
plt.figure(figsize=(10, 6))
plt.plot(epochs, learning_rate, label='training', color='limegreen')
plt.scatter(epochs[0], learning_rate[0], color='limegreen')  # Highlight the start point

# Title and labels
plt.title("Gemma 2B Model Learning Rate Training", fontsize=14, fontweight='bold')
plt.xlabel("Epochs", fontsize=12)
plt.ylabel("Learning Rate", fontsize=12)

# Customizing the axes
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.ylim(1.5e-4, 2.5e-4)  # Compressing the y-axis to fit within specific coordinates
plt.grid(True, which='both', linestyle='--', linewidth=0.5)

# Adding a legend
plt.legend(loc='upper right', fontsize=10, frameon=True, title_fontsize='12')

# Arrow annotations
plt.annotate('', xy=(10, 2e-4), xytext=(10, 2.4e-4),
             arrowprops=dict(facecolor='black', arrowstyle='wedge,tail_width=0.7'))
plt.text(5, 2.45e-4, 'Learning Rate', fontsize=10, rotation=90)
plt.annotate('', xy=(90, 1.6e-4), xytext=(110, 1.6e-4),
             arrowprops=dict(facecolor='black', arrowstyle='wedge,tail_width=0.7'))
plt.text(95, 1.55e-4, 'Epochs', fontsize=10)

# Show the plot
plt.show()

In [ ]:
!pip install gradio
import gradio as gr

# Placeholder for the `queries` function
def queries(message):
    return f"Response to: {message}"

# Build Gradio App
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown(
        """
        # Welcome to Fine-Tuned LoRA Application with Gemma 2B!
        Start interacting with your chatbot companion.
        """
    )
    chatbot = gr.Chatbot()
    msg = gr.Textbox(label="Your Message", placeholder="Type your message here...")

    # Clear button
    clear = gr.Button("Clear Chat")

    # Respond function
    def respond(message, chat_history):
        bot_message = queries(message)
        chat_history.append((message, bot_message))
        return "", chat_history

    # Clear chat function
    def clear_chat():
        return [], ""

    # Event handling
    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    clear.click(clear_chat, [], [chatbot, msg])

# Launch Gradio App
demo.launch(share=True)

In [ ]:
# gemma_lm.backbone.save("gemma_model.keras")  # Change the filepath to include .keras extension

In [ ]:
#!pip install streamlit -q

write the cell python code into an app.py file


In [ ]:
#%%writefile app.py
#import streamlit as st
#from typing import List, Tuple
#import keras

#import keras_nlp

# import subprocess
# import sys

# # Function to install a library
# def install_library(library_name):
#     subprocess.check_call([sys.executable, "-m", "pip", "install", library_name])

# # Example: Install `keras-nlp` and `streamlit`
# install_library("keras-nlp")
# # Load the backbone of the GEMMA model

#from keras.models import load_model

#gemma_lm_backbone = load_model("gemma_model.keras")

# Rebuild the GEMMA model with the loaded backbone
#gemma_lm = keras_nlp.models.GemmaCausalLM(backbone=gemma_lm_backbone)

# Configure the model (e.g., sampler, preprocessor settings) if needed
#sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
#gemma_lm.compile(sampler=sampler)
#gemma_lm.preprocessor.sequence_length = 256


# Define a function for generating responses (mockup of the "queries" function)
#def queries(Instruction: str) -> str:
#    template = "Instruction:\n{instruction}\n\nResponse:\n{response}"
#    prompt = template.format(
#      instruction=Instruction,
#      response="",
#    )

 #   return f"Response to: {gemma_lm.generate(prompt, max_length=256)}"

# Define a function to manage chatbot interactions
#def respond(Instruction: str, chat_history: List[Tuple[str, str]]) -> List[Tuple[str, str]]:
#    Response = queries(Instruction)
#    chat_history.append((Instruction, Response))
#    return chat_history

# Streamlit UI|
#st.set_page_config(page_title="Fine-Tuned LoRA Application with Gemma 2B", layout="wide")

#st.markdown("""
    # Welcome to Fine-Tuned LoRA Application with Gemma 2B!
#   Start interacting with your chatbot companion.
#""")

# Initialize chat history in session state
#if "chat_history" not in st.session_state:
#    st.session_state.chat_history = []

# Chatbot UI
#for Instruction, Response in st.session_state.chat_history:
#    st.markdown(f"**User:** {Instruction}")
#    st.markdown(f"**Gemma 2B:** {Response}")

# User input
#Instruction = st.text_input("Ask your question:", key="Instruction")
#if st.button("Send") or st.session_state.get("Response"):
#    if Instruction.strip():
#        st.session_state.chat_history = respond(Instruction, st.session_state.chat_history)


# Clear button
#if st.button("Clear Chat"):
#    st.session_state.chat_history = []



In [ ]:
#!streamlit run app.py & npx localtunnel --port 8501